# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 2 2022 2:16PM,243296,10,8352279,"New Skincare Co, LLC dba Covey",Released
1,Aug 2 2022 1:47PM,243295,10,0085919215,ISDIN Corporation,Released
2,Aug 2 2022 1:47PM,243295,10,0085921523,ISDIN Corporation,Released
3,Aug 2 2022 1:47PM,243295,10,0085921527,ISDIN Corporation,Released
4,Aug 2 2022 1:47PM,243295,10,0085921525,ISDIN Corporation,Released
5,Aug 2 2022 1:47PM,243295,10,0085921526,ISDIN Corporation,Released
6,Aug 2 2022 1:47PM,243295,10,0085921524,ISDIN Corporation,Released
7,Aug 2 2022 1:47PM,243295,10,0085921528,ISDIN Corporation,Released
8,Aug 2 2022 1:47PM,243295,10,0085921541,ISDIN Corporation,Released
9,Aug 2 2022 1:47PM,243295,10,0085921549,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
48,243292,Released,2
49,243293,Released,5
50,243294,Released,2
51,243295,Released,21
52,243296,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243292,NaN,NaN,2.0
243293,NaN,NaN,5.0
243294,NaN,NaN,2.0
243295,NaN,NaN,21.0
243296,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243173,0.0,13.0,3.0
243174,7.0,12.0,18.0
243204,4.0,15.0,0.0
243208,0.0,17.0,2.0
243210,0.0,11.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243173,0,13,3
243174,7,12,18
243204,4,15,0
243208,0,17,2
243210,0,11,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243173,0,13,3
1,243174,7,12,18
2,243204,4,15,0
3,243208,0,17,2
4,243210,0,11,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243173,,13,3
1,243174,7,12,18
2,243204,4,15,
3,243208,,17,2
4,243210,,11,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey"
1,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation
22,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc."
23,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV"
24,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation
29,Aug 2 2022 1:35PM,243292,10,Emerginnova
31,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC"
33,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc."
36,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc."
44,Aug 2 2022 1:08PM,243288,10,Yusen – 3D Matrix


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",,,1
1,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,,,21
2,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",,,2
3,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",,,2
4,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,,,5
5,Aug 2 2022 1:35PM,243292,10,Emerginnova,,,2
6,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",,,2
7,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",,,3
8,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",,,8
9,Aug 2 2022 1:08PM,243288,10,Yusen – 3D Matrix,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",1,,
1,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,21,,
2,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",2,,
3,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",2,,
4,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5,,
5,Aug 2 2022 1:35PM,243292,10,Emerginnova,2,,
6,Aug 2 2022 1:33PM,243291,10,"Snap Medical Industries, LLC",2,,
7,Aug 2 2022 1:22PM,243289,10,"ACI Healthcare USA, Inc.",3,,
8,Aug 2 2022 1:10PM,243286,20,"Exact-Rx, Inc.",8,,
9,Aug 2 2022 1:08PM,243288,10,Yusen – 3D Matrix,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",1,,
1,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,21,,
2,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",2,,
3,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",2,,
4,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",1.0,NaN,NaN
1,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,21.0,NaN,NaN
2,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",2.0,NaN,NaN
3,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",2.0,NaN,NaN
4,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 2 2022 2:16PM,243296,10,"New Skincare Co, LLC dba Covey",1.0,0.0,0.0
1,Aug 2 2022 1:47PM,243295,10,ISDIN Corporation,21.0,0.0,0.0
2,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc.",2.0,0.0,0.0
3,Aug 2 2022 1:45PM,243294,10,"Blinc International, Inc. - PV",2.0,0.0,0.0
4,Aug 2 2022 1:39PM,243293,10,ISDIN Corporation,5.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,5108827,123.0,19.0,0.0
15,486347,21.0,25.0,7.0
16,243278,0.0,1.0,0.0
19,1945909,6.0,18.0,4.0
20,1702663,34.0,49.0,14.0
21,729737,1.0,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,5108827,123.0,19.0,0.0
1,15,486347,21.0,25.0,7.0
2,16,243278,0.0,1.0,0.0
3,19,1945909,6.0,18.0,4.0
4,20,1702663,34.0,49.0,14.0
5,21,729737,1.0,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,123.0,19.0,0.0
1,15,21.0,25.0,7.0
2,16,0.0,1.0,0.0
3,19,6.0,18.0,4.0
4,20,34.0,49.0,14.0
5,21,1.0,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,123.0
1,15,Released,21.0
2,16,Released,0.0
3,19,Released,6.0
4,20,Released,34.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,7.0,0.0,4.0,14.0,0.0
Executing,19.0,25.0,1.0,18.0,49.0,2.0
Released,123.0,21.0,0.0,6.0,34.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,7.0,0.0,4.0,14.0,0.0
1,Executing,19.0,25.0,1.0,18.0,49.0,2.0
2,Released,123.0,21.0,0.0,6.0,34.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,7.0,0.0,4.0,14.0,0.0
1,Executing,19.0,25.0,1.0,18.0,49.0,2.0
2,Released,123.0,21.0,0.0,6.0,34.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()